In [34]:
import scipy.io
import tensorflow as tf
import numpy as np
from tensorflow import  keras
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout

In [35]:
def load_data():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["data"].transpose()  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['data'].transpose() # (262134, 270) 
    myopathy = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['data'].transpose() #(262134, 315)
    return als,normal,myopathy

In [36]:

def scale_data_minmax(data):
#type of scaler that scales the minimum and maximum values to be 0 and 1 respectively. 
    scaler = MinMaxScaler()
#fit_transform() then it will calculate the mean(μ) and standard deviation(σ) of the feature F at a time it will transform the data points of the feature F.    
    normalized_data =scaler.fit_transform(data)


    return normalized_data

In [37]:
def scale_data_standard(data):
    scaled_data = scale(data,axis=1)
    return scaled_data

In [38]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
        arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
        arg = tf.reshape(arg,dim)
        dataset.append(arg)

    return dataset

In [190]:
def windowing_fun(data,window_size=1000,overlap=100):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data


In [40]:
def load_file_names():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["files"]  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['files'] # (262134, 270) 
    myopathy = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['files'] #(262134, 315)
    return als,normal,myopathy

In [41]:
def generate_dataframe(data,files):
    df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
    windowed_data = [{"signal": data[j],"type_signal":files[0][j][0][5],"num_personne": files[0][j][0][6:8],"muscle": files[0][j][0][8:10],"num_enregistrement":files[0][j][0][10:12] } for j in range(0,len(data)) ]
    final_data = pd.DataFrame(windowed_data)
    #df =df.append(final_data, ignore_index=True)
    #df =pd.concat([df, final_data])
    df = pd.concat([df, final_data], ignore_index=True)
    return df

In [159]:
als, normal , myopathy = load_data()

In [160]:
alsfiles,normalfiles,myopathyfiles = load_file_names()

In [161]:
df = generate_dataframe(np.vstack((normal,als,myopathy)),np.concatenate((normalfiles,alsfiles,myopathyfiles),axis=1))

In [162]:
def f(row):
    return {"C":0,"A":1,"M":2}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [163]:
df = df.query("muscle=='BB'")

In [164]:
X = df['signal'].tolist()
Y = df['num_class'].tolist()

In [165]:
X=X[170:]
Y=Y[170:]

In [166]:
del df

In [ ]:
##WAVEFORM WRAP
# Select a few random samples from your dataset
samples = random.sample(list(X), k=3)

def waveform_warp(signal, intensity):
    t = np.arange(len(signal))
    random_shift = np.random.uniform(-intensity, intensity)
    warping_function = np.sin(t + random_shift)
    warping_function = np.reshape(warping_function, (len(warping_function), 1))
    warped_signal = signal * warping_function
    return warped_signal

# Plot the original and augmented signals
for sample in samples:
    fig, axs = plt.subplots(2, 1, figsize=(10, 5))
    axs[0].plot(sample)
    axs[0].set_title('Original Signal')
    
    # Apply augmentation techniques
    X = np.array(X)
    augmented_sample = waveform_warp(X, 2)
    
    
    axs[1].plot(augmented_sample)
    axs[1].set_title('Augmented Signal')
    
    plt.show()


In [167]:
np.shape(X)

(305, 262134)

In [168]:
#data augmentation using waveform wrap
def waveform_warp(signal, intensity):
    t = np.arange(len(signal))
    random_shift = np.random.uniform(-intensity, intensity)
    warping_function = np.sin(t + random_shift)
    warping_function = np.reshape(warping_function, (len(warping_function), 1))
    warped_signal = signal * warping_function
    return warped_signal

In [169]:
X = np.array(X)
np.shape(X)

(305, 262134)

In [170]:
augmented_X = waveform_warp(X, 2)
np.shape(X)

(305, 262134)

In [171]:
data = [*augmented_X]
np.shape(data)

(305, 262134)

In [172]:
np.shape(augmented_X)

(305, 262134)

In [173]:
data = [*X]
#data = data + X

In [174]:
augmented_data = np.concatenate((data, augmented_X))
np.shape(augmented_data)

(610, 262134)

In [175]:
del  als,normal,myopathy,alsfiles,normalfiles,myopathyfiles 

In [176]:
data = augmented_data
data = scale_data_standard(data)
np.shape(data), np.shape(X)

((610, 262134), (305, 262134))

In [185]:
np.shape(data)

(610, 262134)

In [186]:
count_normal = Y.count(0)
last_index_normal = count_normal * 2 - 1
last_index_normal

199

In [189]:
count_als = Y.count(1)
last_index_als = last_index_normal + count_als * 2
last_index_als

395

In [188]:
count_myopathy = Y.count(2)
last_index_myopathy = last_index_als + count_myopathy * 2
last_index_myopathy

609

In [191]:
window_size = 10000
overlap = 1000
X_windowed = windowing_fun(X, window_size, overlap)

In [192]:
count_normal = np.count_nonzero(Y == 0)
count_als = np.count_nonzero(Y == 1)
count_myopathy = np.count_nonzero(Y == 2)

In [195]:
Y_windowed = [0]*count_normal + [1]*count_als + [2]*count_myopathy

In [197]:
# remove excess windows for which the label is unknown
num_samples = min(len(X_windowed), len(Y_windowed))
X_windowed = X_windowed[:num_samples]
Y_windowed = Y_windowed[:num_samples]

In [198]:
X_windowed = np.array(X_windowed)

In [199]:
# shuffle the data and labels in unison
rng = np.random.default_rng()
indices = rng.permutation(num_samples)
X_windowed = X_windowed[indices]
Y_windowed = np.array(Y_windowed)[indices]

In [200]:
np.shape(Y)

(305,)

In [155]:
X = windowing_fun(data,10000)

In [201]:
np.shape(X)

(305, 262134)

In [202]:
del data

In [203]:
X = scale_data_standard(X)

In [205]:
tensors_list = gen_tensors_list(X,(1,262134))

In [207]:
#X_train, X_test, y_train, y_test = train_test_split( tensors_list, Y, test_size=0.04,random_state=20)
X_train, X_val, y_train, y_val = train_test_split(tensors_list, Y, test_size=0.2, random_state=42)
np.shape(X_train), np.shape(X_val)

((244, 1, 262134), (61, 1, 262134))

In [208]:
model = Sequential([
    #Conv2D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(64, 5, activation = 'relu', padding ="same", input_shape = (1,262134)), 
    
    #MaxPooling2D(filter size stride and pad can also be added)
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #LAYER2
    Conv1D(32, 5, activation = 'relu', padding ="same"), #inc filter size and stride
    MaxPooling1D(2, strides = 2, padding ="same"),
    
    #FLATTEN
    Flatten(),
    
    #FC Layers:
    
    #LAYER3
    #Dense : fully connected  Dense(#neurons, activation function )
    Dense(64, activation = 'relu'),
    Dropout(0.6),
    
    #LAYER4
    Dense(3, activation = 'softmax'),
    
    
])


In [209]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [211]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

In [212]:
#model.fit(X_train, y_train, epochs =20, batch_size = 128)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
history = model.fit(
                    X_train, 
                    y_train, 
                    validation_data=(X_val, y_val), 
                    validation_split=0.1,
                    epochs=60, 
                    batch_size=64)


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


Epoch 1/60
4/4 [==============================] - 45s 11s/step - loss: 1.7186 - accuracy: 0.7910 - val_loss: 3.1445 - val_accuracy: 0.4098
Epoch 2/60
4/4 [==============================] - 39s 10s/step - loss: 1.3471 - accuracy: 0.8320 - val_loss: 3.0537 - val_accuracy: 0.3770
Epoch 3/60
4/4 [==============================] - 39s 10s/step - loss: 1.3623 - accuracy: 0.7746 - val_loss: 3.0406 - val_accuracy: 0.3607
Epoch 4/60
4/4 [==============================] - 43s 11s/step - loss: 1.3106 - accuracy: 0.8115 - val_loss: 2.9507 - val_accuracy: 0.3607
Epoch 5/60
4/4 [==============================] - 39s 10s/step - loss: 1.4417 - accuracy: 0.8156 - val_loss: 2.7842 - val_accuracy: 0.3607
Epoch 6/60
4/4 [==============================] - 39s 10s/step - loss: 0.9262 - accuracy: 0.8648 - val_loss: 2.7406 - val_accuracy: 0.3770
Epoch 7/60
4/4 [==============================] - 39s 10s/step - loss: 1.2296 - accuracy: 0.8607 - val_loss: 2.6717 - val_accuracy: 0.3770
Epoch 8/60
4/4 [===========

In [ ]:
score = model.evaluate(np.asarray(X_val),np.asarray (y_val))

In [59]:
model.save("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [60]:
import json

with open('/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D_score.json', 'w+') as file:

    json.dump(score, file)

In [61]:
model= tf.keras.models.load_model("/content/drive/MyDrive/PFE_RESULTS_DATA/augmentation0/scaling/CNN1D/CNN1D.h5")

In [62]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True
    )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [63]:
y_pred = model.predict(np.asarray(X_test))

40/40 [==============================] - 0s 4ms/step


In [64]:
y_predicted_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathy"}[i] for i in np.argmax(y_pred,axis=1)]

In [65]:
y_true_labels = [{0:"Normal",1:"Neuropathie",2:"Myopathy"}[i] for i in y_test]

In [66]:
import matplotlib.pyplot as plt
import seaborn as sns

con_mat = tf.math.confusion_matrix(labels=y_test, predictions=np.argmax(y_pred,axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = ["Normal","Neuropathie"], 
                     columns = ["Normal","Neuropathie"])

figure = plt.figure(figsize=(5,5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


ValueError: Shape of passed values is (3, 3), indices imply (2, 2)